In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# We are investigating the predictivity of central bank policy rates and stock market indices as they relate to the price of gold. 

## Consolidate Central Bank Rates

In [2]:
# Start by consolidating central bank rate data

%store -r df_aud
%store -r df_eur
%store -r df_gbp
%store -r df_usd
%store -r df_yen

In [3]:
df_aud['date'].min()

Timestamp('1990-01-23 00:00:00')

In [4]:
df_eur['date'].min()

Timestamp('1999-01-01 00:00:00')

In [5]:
df_gbp['date'].min()

Timestamp('1975-01-20 00:00:00')

In [6]:
df_usd['date'].min()

Timestamp('1990-01-01 00:00:00')

In [7]:
df_yen['date'].min()

Timestamp('2000-01-01 00:00:00')

In [8]:
# loop through all dataframes and merge via outer join, ensuring the df with the most non-null dates is the left object
df_rates = pd.merge(df_gbp, df_usd, how = 'outer', on = ['date'])
df_rates = pd.merge(df_rates, df_eur, how = 'outer', on = ['date'])
df_rates = pd.merge(df_rates, df_aud, how = 'outer', on = ['date'])
df_rates = pd.merge(df_rates, df_yen, how = 'outer', on = ['date'])

In [9]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11273 entries, 0 to 11272
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           11273 non-null  datetime64[ns]
 1   Rate           238 non-null    float64       
 2   usd_rate_tgt   11109 non-null  float64       
 3   eur_rate       49 non-null     float64       
 4   AUD_rate_tgt%  340 non-null    float64       
 5   yen_rate       3839 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 616.5 KB


In [10]:
df_rates.rename(columns = {'AUD_rate_tgt%': 'aud_rate', 'Rate': 'gbp_rate', 'usd_rate_tgt':'usd_rate'}, inplace = True)

In [11]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11273 entries, 0 to 11272
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      11273 non-null  datetime64[ns]
 1   gbp_rate  238 non-null    float64       
 2   usd_rate  11109 non-null  float64       
 3   eur_rate  49 non-null     float64       
 4   aud_rate  340 non-null    float64       
 5   yen_rate  3839 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 616.5 KB


In [12]:
df_rates = df_rates.sort_values(by = ['date'])

In [13]:
#lowest minimum date across rate data is 2000-01-01 (df_yen), so drop all data prior
df_rates = df_rates[df_rates['date']>'1999-12-31']

In [14]:
df_rates = df_rates.set_index('date')

In [15]:
df_rates.head()

,gbp_rate,usd_rate,eur_rate,aud_rate,yen_rate
date,,,,,
2000-01-01,NaN,5.5,NaN,NaN,NaN
2000-01-02,NaN,5.5,NaN,NaN,NaN
2000-01-03,NaN,5.5,NaN,NaN,NaN
2000-01-04,NaN,5.5,NaN,NaN,NaN
2000-01-05,NaN,5.5,NaN,NaN,NaN


In [21]:
# add up to 10 day lag for all rates

for col in df_rates.columns:
    for i in range(1,11):
        df_rates[f'{col}_lag{i}'] = df_rates[col].shift(i)

In [22]:
df_rates.dropna(inplace = True)

In [24]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7234 entries, 2000-08-11 to 2020-05-31
Data columns (total 55 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gbp_rate        7234 non-null   object 
 1   usd_rate        7234 non-null   float64
 2   eur_rate        7234 non-null   object 
 3   aud_rate        7234 non-null   object 
 4   yen_rate        7234 non-null   object 
 5   gbp_rate_lag1   7234 non-null   object 
 6   gbp_rate_lag2   7234 non-null   object 
 7   gbp_rate_lag3   7234 non-null   object 
 8   gbp_rate_lag4   7234 non-null   object 
 9   gbp_rate_lag5   7234 non-null   object 
 10  gbp_rate_lag6   7234 non-null   object 
 11  gbp_rate_lag7   7234 non-null   object 
 12  gbp_rate_lag8   7234 non-null   object 
 13  gbp_rate_lag9   7234 non-null   object 
 14  gbp_rate_lag10  7234 non-null   object 
 15  usd_rate_lag1   7234 non-null   float64
 16  usd_rate_lag2   7234 non-null   float64
 17  usd_rate_lag3  

In [25]:
for col in d.keys():
    df_rates[col] = df_rates[col].astype('float')

In [26]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7234 entries, 2000-08-11 to 2020-05-31
Data columns (total 55 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   gbp_rate        7234 non-null   float64
 1   usd_rate        7234 non-null   float64
 2   eur_rate        7234 non-null   float64
 3   aud_rate        7234 non-null   float64
 4   yen_rate        7234 non-null   float64
 5   gbp_rate_lag1   7234 non-null   object 
 6   gbp_rate_lag2   7234 non-null   object 
 7   gbp_rate_lag3   7234 non-null   object 
 8   gbp_rate_lag4   7234 non-null   object 
 9   gbp_rate_lag5   7234 non-null   object 
 10  gbp_rate_lag6   7234 non-null   object 
 11  gbp_rate_lag7   7234 non-null   object 
 12  gbp_rate_lag8   7234 non-null   object 
 13  gbp_rate_lag9   7234 non-null   object 
 14  gbp_rate_lag10  7234 non-null   object 
 15  usd_rate_lag1   7234 non-null   float64
 16  usd_rate_lag2   7234 non-null   float64
 17  usd_rate_lag3  

In [27]:
df_rates.head()

,gbp_rate,usd_rate,eur_rate,aud_rate,yen_rate,gbp_rate_lag1,gbp_rate_lag2,gbp_rate_lag3,gbp_rate_lag4,gbp_rate_lag5,...,yen_rate_lag1,yen_rate_lag2,yen_rate_lag3,yen_rate_lag4,yen_rate_lag5,yen_rate_lag6,yen_rate_lag7,yen_rate_lag8,yen_rate_lag9,yen_rate_lag10
date,,,,,,,,,,,,,,,,,,,,,
2000-08-11,6.0,6.5,3.25,6.25,0.25,6,6,6,6,6,...,~,~,~,~,~,~,~,~,~,~
2000-08-12,6.0,6.5,3.25,6.25,0.25,6,6,6,6,6,...,0.25,~,~,~,~,~,~,~,~,~
2000-08-13,6.0,6.5,3.25,6.25,0.25,6,6,6,6,6,...,0.25,0.25,~,~,~,~,~,~,~,~
2000-08-14,6.0,6.5,3.25,6.25,0.25,6,6,6,6,6,...,0.25,0.25,0.25,~,~,~,~,~,~,~
2000-08-15,6.0,6.5,3.25,6.25,0.25,6,6,6,6,6,...,0.25,0.25,0.25,0.25,~,~,~,~,~,~


In [28]:
%store df_rates

Stored 'df_rates' (DataFrame)


## Consolidate stock indices

In [29]:
%store -r df_hsi
%store -r df_sse
%store -r df_jeg
%store -r df_nasdaq
%store -r df_nyse

In [30]:
df_hsi['date'].min()

Timestamp('2000-01-03 00:00:00')

In [31]:
df_sse['date'].min()

Timestamp('1997-07-02 00:00:00')

In [32]:
df_jeg['date'].min()

Timestamp('2008-09-05 00:00:00')

In [33]:
df_nasdaq['date'].min()

Timestamp('2000-01-03 00:00:00')

In [34]:
df_nyse['date'].min()

Timestamp('2000-01-03 00:00:00')

In [35]:
# loop through all dataframes and merge via outer join, ensuring the df with the most non-null dates is the left object
df_stock = pd.merge(df_sse, df_hsi, how = 'outer', on = ['date'])
df_stock = pd.merge(df_stock, df_nyse, how = 'outer', on = ['date'])
df_stock = pd.merge(df_stock, df_nasdaq, how = 'outer', on = ['date'])
df_stock = pd.merge(df_stock, df_jeg, how = 'outer', on = ['date'])

In [36]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5964 entries, 0 to 5963
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          5964 non-null   datetime64[ns]
 1   sse_close     5714 non-null   float64       
 2   hsi_close     5023 non-null   float64       
 3   nyse_close    5131 non-null   float64       
 4   nasdaq_close  5131 non-null   float64       
 5   jeg_close     2869 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 326.2 KB


In [37]:
df_stock = df_stock.sort_values(by = ['date'])

In [38]:
#lowest minimum date across stock data is 2008-09-05 (df_jeg), so drop all data prior
df_stock = df_stock[df_stock['date']>'2008-09-04']

In [39]:
df_stock = df_stock.set_index('date')

In [40]:
df_stock.head()

,sse_close,hsi_close,nyse_close,nasdaq_close,jeg_close
date,,,,,
2008-09-05,2202.446045,19933.279297,8033.759766,2255.879883,335.000000
2008-09-08,2143.420898,20794.269531,8168.620117,2269.760010,343.000000
2008-09-09,2145.779053,20491.109375,7871.149902,2209.810059,337.864990
2008-09-10,2150.759033,19999.779297,7957.259766,2228.699951,319.382996
2008-09-11,2078.980957,19388.720703,8011.250000,2258.219971,290.799988


In [41]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3051 entries, 2008-09-05 to 2020-05-27
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sse_close     2845 non-null   float64
 1   hsi_close     2881 non-null   float64
 2   nyse_close    2950 non-null   float64
 3   nasdaq_close  2950 non-null   float64
 4   jeg_close     2869 non-null   float64
dtypes: float64(5)
memory usage: 143.0 KB


In [45]:
# add up to 10 day lag for all rates
for col in df_stock.columns:
    for i in range(1,11):
        df_stock[f'{col}_lag{i}'] = df_stock[col].shift(i)

In [46]:
df_stock.dropna(inplace = True)

In [47]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3041 entries, 2008-09-19 to 2020-05-27
Data columns (total 55 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sse_close           3041 non-null   float64
 1   hsi_close           3041 non-null   float64
 2   nyse_close          3041 non-null   float64
 3   nasdaq_close        3041 non-null   float64
 4   jeg_close           3041 non-null   float64
 5   sse_close_lag1      3041 non-null   float64
 6   sse_close_lag2      3041 non-null   float64
 7   sse_close_lag3      3041 non-null   float64
 8   sse_close_lag4      3041 non-null   float64
 9   sse_close_lag5      3041 non-null   float64
 10  sse_close_lag6      3041 non-null   float64
 11  sse_close_lag7      3041 non-null   float64
 12  sse_close_lag8      3041 non-null   float64
 13  sse_close_lag9      3041 non-null   float64
 14  sse_close_lag10     3041 non-null   float64
 15  hsi_close_lag1      3041 non-null   f

In [48]:
df_stock.head()

,sse_close,hsi_close,nyse_close,nasdaq_close,jeg_close,sse_close_lag1,sse_close_lag2,sse_close_lag3,sse_close_lag4,sse_close_lag5,...,jeg_close_lag1,jeg_close_lag2,jeg_close_lag3,jeg_close_lag4,jeg_close_lag5,jeg_close_lag6,jeg_close_lag7,jeg_close_lag8,jeg_close_lag9,jeg_close_lag10
date,,,,,,,,,,,,,,,,,,,,,
2008-09-19,2075.091064,19327.730469,8187.129883,2273.899902,308.480988,1895.837036,1929.046997,1986.635986,2079.673096,2079.673096,...,287.200012,281.500000,284.500000,296.140991,296.140991,290.799988,319.382996,337.864990,343.000000,335.000000
2008-09-22,2236.409912,19632.199219,7918.609863,2178.979980,327.000000,2075.091064,1895.837036,1929.046997,1986.635986,2079.673096,...,308.480988,287.200012,281.500000,284.500000,296.140991,296.140991,290.799988,319.382996,337.864990,343.000000
2008-09-23,2201.510010,18872.849609,7785.270020,2153.330078,327.000000,2236.409912,2075.091064,1895.837036,1929.046997,1986.635986,...,327.000000,308.480988,287.200012,281.500000,284.500000,296.140991,296.140991,290.799988,319.382996,337.864990
2008-09-24,2216.811035,18961.990234,7768.819824,2155.679932,339.727997,2201.510010,2236.409912,2075.091064,1895.837036,1929.046997,...,327.000000,327.000000,308.480988,287.200012,281.500000,284.500000,296.140991,296.140991,290.799988,319.382996
2008-09-25,2297.500977,18934.429688,7927.870117,2186.570068,327.000000,2216.811035,2201.510010,2236.409912,2075.091064,1895.837036,...,339.727997,327.000000,327.000000,308.480988,287.200012,281.500000,284.500000,296.140991,296.140991,290.799988


In [49]:
%store df_stock

Stored 'df_stock' (DataFrame)


In [50]:
df_feat = pd.merge(df_rates, df_stock, how = 'inner', on = ['date'])

In [51]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3041 entries, 2008-09-19 to 2020-05-27
Columns: 110 entries, gbp_rate to jeg_close_lag10
dtypes: float64(70), object(40)
memory usage: 2.6+ MB


In [52]:
%store df_feat

Stored 'df_feat' (DataFrame)


## Merge in gold data and create final Xy dataframe for model

In [53]:
%store -r df_gold

In [54]:
df_gold.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7845 entries, 3 to 8417
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            7845 non-null   datetime64[ns]
 1   gold_price_usd  7845 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 183.9 KB


In [55]:
df_gold.rename(columns = {'gold_price_usd': 'gold'}, inplace = True)

In [56]:
for i in range(1,11):
    df_gold[f'gold_lag{i}'] = df_gold['gold'].shift(i)

In [57]:
df_gold.head()

,date,gold,gold_lag1,gold_lag2,gold_lag3,gold_lag4,gold_lag5,gold_lag6,gold_lag7,gold_lag8,gold_lag9,gold_lag10
3,1990-12-31,392.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-12-28,392.75,392.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1990-12-27,384.60,392.75,392.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1990-12-26,384.00,384.60,392.75,392.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1990-12-25,389.00,384.00,384.60,392.75,392.5,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df_gold.dropna(inplace = True)

In [59]:
df_gold.head()

,date,gold,gold_lag1,gold_lag2,gold_lag3,gold_lag4,gold_lag5,gold_lag6,gold_lag7,gold_lag8,gold_lag9,gold_lag10
13,1990-12-17,376.50,375.85,384.00,383.40,381.20,383.00,389.0,384.0,384.6,392.75,392.50
14,1990-12-14,375.80,376.50,375.85,384.00,383.40,381.20,383.0,389.0,384.0,384.60,392.75
15,1990-12-13,373.05,375.80,376.50,375.85,384.00,383.40,381.2,383.0,389.0,384.00,384.60
16,1990-12-12,371.50,373.05,375.80,376.50,375.85,384.00,383.4,381.2,383.0,389.00,384.00
17,1990-12-11,372.00,371.50,373.05,375.80,376.50,375.85,384.0,383.4,381.2,383.00,389.00


In [60]:
df_Xy = pd.merge(df_feat, df_gold, how = 'inner', on = ['date'])

In [61]:
df_Xy.head()

,date,gbp_rate,usd_rate,eur_rate,aud_rate,yen_rate,gbp_rate_lag1,gbp_rate_lag2,gbp_rate_lag3,gbp_rate_lag4,...,gold_lag1,gold_lag2,gold_lag3,gold_lag4,gold_lag5,gold_lag6,gold_lag7,gold_lag8,gold_lag9,gold_lag10
0,2008-09-19,5.0,2.0,3.25,7.0,0.5,5,5,5,5,...,900.84,888.65,887.20,873.34,884.20,899.80,872.02,883.80,840.35,830.55
1,2008-09-22,5.0,2.0,3.25,7.0,0.5,5,5,5,5,...,888.65,887.20,873.34,884.20,899.80,872.02,883.80,840.35,830.55,865.15
2,2008-09-23,5.0,2.0,3.25,7.0,0.5,5,5,5,5,...,887.20,873.34,884.20,899.80,872.02,883.80,840.35,830.55,865.15,879.90
3,2008-09-24,5.0,2.0,3.25,7.0,0.5,5,5,5,5,...,873.34,884.20,899.80,872.02,883.80,840.35,830.55,865.15,879.90,897.20
4,2008-09-25,5.0,2.0,3.25,7.0,0.5,5,5,5,5,...,884.20,899.80,872.02,883.80,840.35,830.55,865.15,879.90,897.20,882.98


In [62]:
%store df_Xy

Stored 'df_Xy' (DataFrame)
